<a href="https://colab.research.google.com/github/PapaZeusOne/Automated-Trading-Bot/blob/main/Automating_Stock_Trades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [286]:
import numpy as np 
import pandas as pd
import os, io
import datetime as dt
import pandas_datareader.data as web
import requests

# **TODO**


> Vergleiche mehrere Aktien miteinander



> Sende Push zu der Aktie, mit der besten Bewertung



> Prüfen wie stark der CCI gewachsen ist in den letzten 14 Tagen

# **Weitere Ideen**


> Aktien Sektoren zuordnen und den Bot nach Sektoren abfragen. Der Bot soll dann die stärkste Aktie aus dem Sektor zurückgeben. Ziemlich cool, wenn man weiß das zum Beispiel der Wasserstoffsektor gerade extrem am steigen ist.




> Sobald man sich für eine Aktie entschieden, wäre es ja auch interessant zu wissen, wann man am besten aussteigen sollte. Dann könnte man Abfragen schreiben und über den Bot eine Push rausschicken, ob man eine bestimmte Aktie, die man mit dem Bot teilt, verkaufen soll oder nicht, weil sich gerade der Kurs schlecht entwickelt.





# **Indikatoren**


> **MACD & Signallinie**: Der MACD (Moving Average Convergence/Divergence) wird in der Chartanalyse genutzt um Trends sowie Kauf-und Verkaufssignale zu erkennen. Dabei werden zwei exponentiell gewichtete gleitende Durchschnitte (26 und 12 Tage) berechnet und voneinander subtrahiert. Des Weiteren wird eine Signallinie berechnet. Hierbei handelt es sich um den 9 Tage exponentiell geglätteten Durchschnitt. Ein steigender MACD ist ein Indikator für einen Aufwärtstrend, ein sinkender MACD ein Indikator für einen Abwärtstrend. Schneidet der MACD die Signallinie von unten nach oben, so liegt ein Kaufsignal vor. Wird die Signallinie vom MACD von oben nach unten geschnitten, so liegt ein Verkaufssignal vor.



> **RSI**: Der RSI (Relative-Stärke-Index) ist ein Marktindikator. Er wird berechnet, indem der gleitende Mittelwert der Auf-und Abwärtsveränderungen für einen bestimmten Zeitraum (in der Regel 14 Tage) ermittelt wird. Der RSI kann Werte zwischen 0 und 100 einnehmen. Die Werte 30 und 70 spielen beim RSI eine besondere Rolle: Bei einem Wert unter 30 gilt der Wert als überverkauft. Dies bedeutet, dass eine Kurserholung (Steigerung) in nächster Zeit anstehen könnte. Bei einem Wert über 70 gilt die Aktie als überkauft. Dies ist ein Indiz dafür, dass der Kurs in nächster Zeit fallen könnte.




> **Bollinger Band**: Es existieren eigentlich zwei Bollinger Bänder, die gemeinsam mit dem gleitenden Mittel, dem arithmetischen Mittel der Kurse, dann das sogenannte Bollinger Band bilden. Mit Hilfe des Bollinger Bands werden der Verlauf der letzten Tage eines Wertpapieres und dessen Trend visualisiert. Man sieht, wo der Kurs durchschnittlich lag und wie stark er an welchen Tagen vom Mittelwert abwich. In der Regel verwendet man dafür den Zeitraum der letzten 20 Tage.





> **Bollinger Bands oder Keltner Channels?** Schau hier warum die BB die bessere Wahl wären: https://tradeciety.com/keltner-channel-vs-bollinger-bands-one-better/ Aber statt der Verwendung der Standardabweichung zur Darstellung der Bollinger Bänder wird die Average True Range von Wilders eingesetzt. Dieser Unterschied bewirkt, dass der Keltner Channel ruhiger und gleichmäßiger aussieht als die Bollinger Bänder.



> **EMA**: Exponential Moving Average verwendet man, um Kursschwankungen zu glätten und den zugrunde liegenden Trend dadurch besser zu erkennen. Hierzu betrachtet man einen Kursverlauf für einen bestimmten Zeitraum in der Vergangenheit und berechnet daraus den Durchschnitt. **Formel**: https://admiralmarkets.com/de/wissen/articles/forex-indicators/ema-der-exponentiell-gleitende-durchschnitt-als-universelles-hilfsmittel-fur-ihr-trading




> **Commodity Channel Index (CCI)**: Für den CCI wird die Abweichung zwischen Kurs und dessen Gleitenden Durchschnitt herangezogen, um das Momentum zu erfassen. Um dies zu bewerkstelligen, wird die Abweichung zwischen dem aktuellen Betrachtungszeitraum und der durchschnittlichen Abweichung der vergangenen X Zeitperioden verglichen. Im Gegensatz zum Stochastik-Oszillator und dem RSI sind die Werte des CCI nicht begrenzt. Aber in der Praxis bewegt sich der CCI aufgrund seiner Konstruktion zwischen -100 und +100. 
Bemerkenswerterweise werden beim CCI nicht die Schlusskurse, sondern die typischen Kurse (Hoch + Tief + Schlusskurs / 3) zugrunde gelegt. Dann wird die Abweichung bzw. Entfernung zwischen typischem Kurs und Gleitendem Durchschnitt ermittelt. **Formel**: https://www.kagels-trading.de/technische-indikatoren-trader/ **WICHTIG**: Wie der Name schon sagt, gründet der CCI auf der Annahme, dass sich die Kurse normalerweise innerhalb eines unsichtbaren Kanals um den Gleitenden Durchschnitt herum aufhalten.














# **Indikatoren mit denen wir anfangen sollten**


> 1. EMA


> 2. MACD


> 3. Bollinger Bänder



> 4. RSI












In [287]:
#indicator = pd.DataFrame(df, index=df.index)

In [288]:
def calc_indicators(df):
  df['ema_25'] = calc_ema_25(df)
  df['ema_100'] = calc_ema_100(df)

  df['sma_25'] = calc_sma_25(df)
  df['sma_100'] = calc_sma_100(df)

  calc_bollinger(df)

  calc_macd_signalline(df)

  df['RSI'] = calc_rsi(df)

  df['CCI'] = calc_cci(df)

  # df['OBV_MA'] = calc_obv(df)
   
  df = calculate_dead_cat_bounce_v2(df)

  return df

In [289]:
def calc_ema_25(df):
    ema_25 = 25
    df_ema_25 = df['Adj Close'].ewm(span=ema_25, adjust=False).mean()

    return df_ema_25

def calc_ema_100(df):
    ema_100 = 100
    df_ema_100 = df['Adj Close'].ewm(span=ema_100, adjust=False).mean()

    return df_ema_100

In [290]:
def calc_sma_25(df):
    sma_25 = 25
    df_sma_25 = df['Adj Close'].rolling(window=sma_25).mean()

    return df_sma_25

def calc_sma_100(df):
    sma_100 = 100
    df_sma_100 = df['Adj Close'].rolling(window=sma_100).mean()

    return df_sma_100

In [291]:
def calc_bollinger(df):
    bollinger_avg = 20

    short_avg_ma = df['Adj Close'].rolling(window=bollinger_avg).mean()
    short_avg_std = df['Adj Close'].rolling(window=bollinger_avg).std()

    df['bollinger'] = df['Adj Close'].ewm(span=bollinger_avg, adjust=False).mean()
    df['bollinger_lower_band'] = short_avg_ma - (short_avg_std *2)
    df['bollinger_upper_band'] = short_avg_ma + (short_avg_std *2)

In [292]:
def calc_macd_signalline(df):
    ema_short = 12
    ema_long = 26
    signalline_span = 9

    df_ema_short = df['Adj Close'].ewm(span=ema_short, adjust=False).mean()
    df_ema_long = df['Adj Close'].ewm(span=ema_long, adjust=False).mean()
    df['macd'] = df_ema_short - df_ema_long
    df['signalline'] = df['macd'].ewm(span=signalline_span, adjust=False).mean()

In [293]:
def calc_rsi(df, n=14):
    delta = df['Adj Close'].diff()
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    # RolUp = pd.rolling_mean(dUp, n)
    # RolDown = pd.rolling_mean(dDown, n).abs()

    # Calculate the EWMA
    RolUp = dUp.ewm(span=n).mean()
    RolDown = dDown.abs().ewm(span=n).mean()

    # Calculate the RSI based on EWMA
    RS = RolUp / RolDown
    RSI = 100 - (100/(1+RS))

    # Calculate the SMA
    roll_up2 = dUp.rolling(n).mean()
    roll_down2 = dDown.abs().rolling(n).mean()

    # Calculate the RSI based on SMA
    RS2 = roll_up2 / roll_down2
    RSI2 = 100.0 - (100.0 / (1.0 + RS2))

    own_RSI=(RSI+RSI2)/2
    return own_RSI

In [294]:
def calc_cci(df, n=14, constant=0.015):
    tippingPrice = (df['High'] + df['Low'] + df['Adj Close']) / 3
    tippingPrice_mean = tippingPrice.rolling(n).mean()
    tippingPrice_std = tippingPrice.rolling(n).std()
    
    CCI = pd.Series((tippingPrice - tippingPrice_mean) / (constant * tippingPrice_std)) 
    return CCI

In [295]:
### To Be Adjusted

def calc_obv(df, n = 20):
    i = 0
    OBV = [0]

    while i < df.index[-1]:
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] > 0:
            OBV.append(df.loc[i + 1, 'Volume'])
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] == 0:
            OBV.append(0)
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] < 0:
            OBV.append(-df.loc[i + 1, 'Volume'])
        i = i + 1
    OBV = pd.Series(OBV)
    OBV_ma = pd.Series(OBV.rolling(n, min_periods=n).mean())

    return OBV_ma

In [296]:
def calculate_dead_cat_bounce_v2(df):
  set_freq = '2D'

  df['body_mid'] = (df['Adj Close'] + df['Open'])/2

  pct_change_close = df['body_mid'].pct_change(freq=set_freq, fill_method='ffill')
  pct_change_close.dropna(inplace=True)

  dead_cat_bounce = np.where((df.signalline > df.macd) &
                            ((pct_change_close >= 0.05) | (pct_change_close <= -0.05)) &
                            ((df.Open < df.bollinger_lower_band)|(df.Close < df.bollinger_lower_band)),
                            True, False)

  df['dead_cat_bounce'] = dead_cat_bounce
  del df['body_mid']

  return df

In [297]:
def combine_stocks_to_dataframe(id_list):
  start = dt.datetime(2019,1,1)
  end = dt.datetime.now()
  #end = dt.datetime(2017,1,1)
  df_list = []

  for id in id_list:
    stock = web.DataReader(id, 'yahoo', start, end) # load data
    stock['stock_id'] = id
    stock = calc_indicators(stock) # calc indicators for every stock
    stock.dropna(inplace=True) # maybe delete this later
    df_list.append(stock)

  df_complete = pd.concat(df_list)
  
  return df_complete

In [298]:
df_complete = combine_stocks_to_dataframe(['AMZN', 'AAPL', 'SAP', 'ACN', 'VOW.DE', 'PLUG', 'MDO.DE', 'PYPL', 'SQ', 'V', 'YNDX'])

In [299]:
df_yd = df_complete.drop_duplicates(subset='stock_id', keep='last')

In [300]:
def compute_score_v2(row):
  score=0
  if row['ema_25']>row['ema_100']:
    score+=1
  if row['macd']>row['signalline']:
    score+=1
  if row['Adj Close']>row['bollinger_upper_band']:
    score-=1
  if ((row['RSI']>=50) & (row['RSI']<=70)) | (row['RSI'] <= 30):
    score+=1
  if row['Adj Close']<row['bollinger_lower_band']:
    score+=1
  if row['CCI']<-100:
      score+=1
  if row['Adj Close']>row['sma_25']:
      score+=1
  return score

In [301]:
def ema_score(row):
    ema_25 = row['ema_25']
    ema_100 = row['ema_100']
    score = (ema_25/ema_100)-1
    # if score < 0:
    #     score=0
    return score

def macd_signalline_score(row): # need to change something here
    # relative change might be distorted if values near by 0
    macd = row['macd']
    signalline = row['signalline']
    adj_close = row['Adj Close']
    score = (macd/adj_close+1)-(signalline/adj_close+1)
    return score

# set weighting when close is lower lower_band or upper upper_band
# to be discussed
def bollinger_score(row):
    lower_band = row['bollinger_lower_band']
    upper_band = row['bollinger_upper_band']
    middle_band = row['bollinger']
    adj_close = row['Adj Close']
    if (adj_close>=middle_band):
        if adj_close>=upper_band:
            score = ((adj_close/upper_band)-1)*(-1)
        else:
            score = ((adj_close/((upper_band-middle_band)*0.5+middle_band))-1)*(-1)
    else:
        if (adj_close<=lower_band):
            score = ((adj_close/lower_band)-1)*(-1)
        else:
            score = (adj_close/middle_band)-1
    return score

def sma_score(row):
    sma_25 = row['sma_25']
    adj_close = row['Adj Close']
    score = (adj_close/sma_25)-1
    return score

def rsi_score(row):
    lower_band = 30
    upper_band = 70
    middle_band = (lower_band + upper_band) / 2 
    middle_band1 = 55
    rsi = row['RSI']

    if (rsi>middle_band):
        if rsi>=upper_band:
            score = ((rsi/upper_band)-1)*(-1)
        else:
            if rsi<middle_band1:
                score = 0
            else:
                score = ((rsi/((upper_band-middle_band1)*0.5+middle_band1))-1)*(-1)
    else:
        if (rsi<=lower_band):
            score = ((rsi/lower_band)-1)*(-1)
        else:
            score = (rsi/middle_band)-1

    return score

# We should check whether we give the same weighting if it hits above or lower 100/-100
def cci_score(row):
    lower_band = -100
    middle_lower_band = -50
    lower_band2 = -150
    upper_band = 100
    middle_upper_band = 50
    upper_band2 = 150
    cci = row['CCI']

    if (cci > 0):
        if (cci >= upper_band2):
            score = (upper_band2/cci)-1
        else:
            if (cci >= upper_band):
                score = ((cci/((upper_band2-upper_band)*0.5+upper_band))-1) * (-1) # scores positive value if cci stays between 100 and 125
            else:
                if(cci>=middle_upper_band):
                    score = ((cci/((upper_band-middle_upper_band)*0.5+middle_upper_band))-1) * (-1)
                else:
                    score = 0 # if cci stays lower 50
    else:
        if (cci<=lower_band2):
            score = (cci/lower_band2)-1 # scores positive if value stays lower -150, strong signal to buy because the paper is oversold
        else:
            if(cci<=lower_band):            
                score = ((cci/((lower_band2 - lower_band)*0.5+lower_band))-1) # score gets better, the further we move to -150
            else:
                if(cci<=middle_lower_band):
                    score = (middle_lower_band/cci)-1 # score negative value, the closer the stock moves to -100
                else:
                    score = 0
    
    return score

In [302]:
def relative_score(row):
    r = row
    stock = row['stock_id']
    ema_scr = ema_score(r)
    macd_signalline_scr = macd_signalline_score(r)
    bollinger_scr = bollinger_score(r)
    sma_scr = sma_score(r)
    rsi_scr = rsi_score(r)
    cci_scr = cci_score(r)

    score = ema_scr + (macd_signalline_scr*10) + bollinger_scr + sma_scr + rsi_scr + cci_scr
    print(stock, ema_scr, (macd_signalline_scr*10), bollinger_scr, sma_scr, rsi_scr, cci_scr, score, '\n')
    return score

In [303]:
df_yd['score'] = df_yd.apply(lambda row: relative_score(row), axis=1)
df_yd.sort_values(by=['score'], ascending=False)

AMZN 0.013235537749841653 -0.030891854018133547 -0.01476374003224401 -0.02797481356182341 -0.22973077207879422 0 -0.29012564194115353 

AAPL 0.01087587845571747 -0.026986328387529346 -0.010258492496176275 -0.02515988427810323 -0.15443226472718818 0 -0.20596109143327956 

SAP 0.040250878735013096 -0.014204401119211063 0.0017483681970015263 0.01044814704342345 0.11057864166399423 0.006851495642883232 0.15567313016310447 

ACN 0.05706562504764334 -0.043208050445551205 -0.0026742045919966317 -0.010772939131080128 -0.11409606972544584 0 -0.11368563884643046 

VOW.DE 0.1233084945012497 0.04018557819113977 0.004314858820454459 0.010048633545499674 0.055155044654504715 0.051338717256863786 0.2843513269697121 

PLUG -0.19874059311043013 0.340934509629397 0.027220410377411963 0.08123812045143208 0.05787255302421823 -0.31445571802121086 -0.005930717649181738 

MDO.DE 0.03295233628303729 -0.034809795814105815 -0.010218755996801332 -0.016105276264516744 -0.31097271966752027 0 -0.3391542114599069 



,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,sma_25,sma_100,bollinger,bollinger_lower_band,bollinger_upper_band,macd,signalline,RSI,CCI,dead_cat_bounce,score
Date,,,,,,,,,,,,,,,,,,,,
2021-05-21,274.399994,270.799988,272.000000,272.600006,35038.0,272.600006,VOW.DE,269.182177,239.633349,269.888000,231.568001,268.668091,253.205423,278.894577,-1.885220,-2.980679,59.052810,118.582660,False,0.284351
2021-05-21,65.370003,64.639999,64.800003,64.940002,407629.0,64.940002,YNDX,63.874072,64.306621,63.584400,65.676900,63.915240,61.605972,66.920029,0.222508,0.089304,55.847542,68.318215,False,0.237936
2021-05-21,140.600006,139.830002,140.320007,139.830795,152823.0,139.830795,SAP,136.669591,131.381375,138.384929,128.910073,137.271393,133.403464,142.880005,1.436687,1.635308,55.588835,74.486138,False,0.155673
2021-05-21,229.309998,226.880005,226.880005,227.919998,2157970.0,227.919998,V,226.137943,217.821338,228.129330,215.963404,226.521448,220.632401,236.446417,0.566541,1.163231,48.875072,20.402429,False,0.003981
2021-05-21,28.065800,26.820000,27.735001,27.849199,14003333.0,27.849199,PLUG,26.774797,33.415891,25.756768,42.300292,26.184897,20.108860,31.072060,-1.079852,-2.029328,58.882965,98.584179,False,-0.005931
2021-05-21,287.140015,284.700012,285.000000,285.209991,428891.0,285.209991,ACN,285.520943,270.107112,288.316002,268.098787,285.974746,280.332034,295.681971,0.282239,1.514576,44.295197,-21.978083,False,-0.113686
2021-05-21,255.210007,250.960999,253.800995,253.020004,2150046.0,253.020004,PYPL,252.003511,247.043354,255.412802,253.796051,251.085186,232.329657,273.649347,-3.134515,-3.208933,51.059741,90.174181,False,-0.153045
2021-05-21,128.000000,126.049896,127.820000,126.260002,28507922.0,126.260002,AAPL,127.802920,126.427905,129.518677,128.802621,127.568664,121.517289,135.621626,-1.041064,-0.700335,42.278387,2.044126,False,-0.205961
2021-05-21,3256.300049,3206.111328,3250.000000,3213.739990,1464138.0,3213.739990,AMZN,3265.277004,3222.623845,3306.231191,3223.586086,3261.897801,3090.789353,3502.924636,-19.995288,-10.067450,38.513461,-21.737788,False,-0.290126


In [304]:
# df_yd['score'] = df_yd.apply(lambda row: compute_score_v2(row), axis=1)
# df_yd

In [305]:
#df_complete[df_complete.dead_cat_bounce==True]

In [306]:
def bot_warn_me(text):
    
    # Change the strings: <TOKEN> and <CHAT_ID>, with your token and chat_id
    bot_token_id = '1790740607:AAE7Omfuea3tG3iii9A1qX-vQJ9pdLFcO0s'
    bot_chat_id = '908941210'
    
    send_text = 'https://api.telegram.org/bot' + bot_token_id + '/sendMessage?chat_id=' + bot_chat_id + '&parse_mode=Markdown&text=' + text
    
    response = requests.get(send_text)

    return response.json()